# 서울시 구별 CCTV 데이터와 인구 합치기

In [51]:
import numpy as np
import pandas as pd

### 1. CCTV 데이터

In [52]:
cctv = pd.read_csv('서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv',
                    skiprows=1, encoding='euc-kr')        # skiprows 건너뛸 행
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289


In [53]:
# Null data가 있는지 확인
cctv.isna().sum().sum()

0

In [54]:
# 1000 단위 구분 기호 없애기
for column in cctv.columns[1:]:
    cctv[column] = cctv[column].apply(lambda x: int(x.replace(',','')))
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289


In [55]:
# 최근 증가율
cctv['최근3개년'] = cctv['2018년'] + cctv['2019년'] + cctv['2021년']
cctv['2017년이전'] = cctv.총계 - cctv['최근3개년'] - cctv['2021년']
cctv['최근증가율'] = (cctv['최근3개년'] / cctv['2017년이전'] * 100).round(2)
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근3개년,2017년이전,최근증가율
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,23445,51979,45.10
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,137,1592,8.61
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,830,1214,68.37


In [56]:
cctv['최근증가율2'] = cctv.iloc[:, 9:12].sum(axis=1) / cctv.iloc[:,2:9].sum(axis=1) * 100
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근3개년,2017년이전,최근증가율,최근증가율2
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,23445,51979,45.10,77.656342
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,137,1592,8.61,20.487805
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,830,1214,68.37,78.984238


In [57]:
# Selection - 필요한 컬럼 추출
cctv = cctv[['구분','총계','최근증가율']]
cctv.head(3)

,구분,총계,최근증가율
0,계,77032,45.10
1,종로구,1772,8.61
2,중 구,2333,68.37


In [58]:
# 컬럼명 변경(rename써서 딕셔너리형태로 넣기 / inplace로 계속 확인해가며 하기)
cctv.rename(columns={'구분':'구별','총계':'CCTV댓수'}, inplace=True)
cctv.head(3)

,구별,CCTV댓수,최근증가율
0,계,77032,45.10
1,종로구,1772,8.61
2,중 구,2333,68.37


In [59]:
# 첫번째 행(계) 삭제
cctv.drop([0], inplace=True)
cctv.head(3)


,구별,CCTV댓수,최근증가율
1,종로구,1772,8.61
2,중 구,2333,68.37
3,용산구,2383,22.14


In [60]:
# 구 이름에서 공백지우기
cctv.구별 = cctv.구별.str.replace(' ','')
cctv.head(3)

,구별,CCTV댓수,최근증가율
1,종로구,1772,8.61
2,중구,2333,68.37
3,용산구,2383,22.14


In [61]:
# CCTV 댓수가 많은 자치구 Top5
cctv.sort_values('CCTV댓수', ascending=False).head()

,구별,CCTV댓수,최근증가율
23,강남구,6502,30.99
21,관악구,4942,37.44
17,구로구,4075,29.24
8,성북구,3958,88.41
12,은평구,3791,99.24


# 2. 서울시 인구 데이터

In [62]:
pop = pd.read_csv('서울시주민등록인구(구별)통계.txt', sep='\t', skiprows=2)
pop.head(3)

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021.2/4,합계,"4,405,638","9,795,426","4,756,178","5,039,248","9,565,990","4,649,442","4,916,548","229,436","106,736","122,700",2.17,"1,576,971"
1,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"
2,2021.2/4,중구,"62,897","132,259","64,526","67,733","123,016","60,019","62,997","9,243","4,507","4,736",1.96,"23,980"


In [63]:
# Null 데이터 확인
pop.isna().sum().sum()

0

In [64]:
# 필요한 컬럼 추출 및 이름 변경
pop = pop[['자치구','계','계.2','65세이상고령자']]
pop.columns = ['구별','인구수','외국인','고령자']
pop.head(3)

,구별,인구수,외국인,고령자
0,합계,"9,795,426","229,436","1,576,971"
1,종로구,"155,106","9,077","27,605"
2,중구,"132,259","9,243","23,980"


In [68]:
# 천단위 구분기호 없애기
for column in pop.columns[1:]:
    pop[column] = pop[column].str.replace(',','').astype(int)
pop.head(3)

,구별,인구수,외국인,고령자
0,합계,9795426,229436,1576971
1,종로구,155106,9077,27605
2,중구,132259,9243,23980


In [69]:
# 첫번째 행 지우기
pop.drop([0], inplace=True)

In [70]:
# 외국인 비율, 고령자 비율 컬럼 만들기
pop['외국인비율'] = (pop.외국인 / pop.인구수 * 100).round(2)
pop['고령자비율'] = (pop.고령자 / pop.인구수 * 100).round(2)
pop.head()

,구별,인구수,외국인,고령자,외국인비율,고령자비율
1,종로구,155106,9077,27605,5.85,17.80
2,중구,132259,9243,23980,6.99,18.13
3,용산구,240665,14287,38884,5.94,16.16
4,성동구,295767,6605,45591,2.23,15.41
5,광진구,355306,12825,50815,3.61,14.30


## 3. 두 개의 데이터 프레임 합치기

In [71]:
df = pd.merge(cctv, pop)
df.head()

,구별,CCTV댓수,최근증가율,인구수,외국인,고령자,외국인비율,고령자비율
0,종로구,1772,8.61,155106,9077,27605,5.85,17.80
1,중구,2333,68.37,132259,9243,23980,6.99,18.13
2,용산구,2383,22.14,240665,14287,38884,5.94,16.16
3,성동구,3602,26.02,295767,6605,45591,2.23,15.41
4,광진구,2588,89.76,355306,12825,50815,3.61,14.30


In [72]:
df.set_index('구별', inplace=True)
df.head()

,CCTV댓수,최근증가율,인구수,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,
종로구,1772,8.61,155106,9077,27605,5.85,17.80
중구,2333,68.37,132259,9243,23980,6.99,18.13
용산구,2383,22.14,240665,14287,38884,5.94,16.16
성동구,3602,26.02,295767,6605,45591,2.23,15.41
광진구,2588,89.76,355306,12825,50815,3.61,14.30


In [73]:
df.to_csv('서울시_CCTV_인구_병합데이터.csv')